## Question 1
calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import scipy.stats as stats
import re

In [2]:
print("re:", re.__version__, "\nnumpy:", np.__version__, "\npandas:", pd.__version__, "\nscipy:", sp.__version__)

re: 2.2.1 
numpy: 1.23.5 
pandas: 1.5.3 
scipy: 1.10.1


In [3]:
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]
cities

,Metropolitan area,Population (2016 est.)[8],NFL,MLB,NBA,NHL
0,New York City,20153634,Giants Jets[note 1],Yankees Mets[note 2],Knicks Nets,Rangers Islanders Devils[note 3]
1,Los Angeles,13310447,Rams Chargers[note 4],Dodgers Angels,Lakers Clippers,Kings Ducks
2,San Francisco Bay Area,6657982,49ers Raiders[note 6],Giants Athletics,Warriors,Sharks[note 7]
3,Chicago,9512999,Bears[note 8],Cubs White Sox,Bulls[note 9],Blackhawks
4,Dallas–Fort Worth,7233323,Cowboys,Rangers,Mavericks,Stars
5,"Washington, D.C.",6131977,Redskins,Nationals[note 10],Wizards[note 11],Capitals
6,Philadelphia,6070500,Eagles,Phillies[note 12],76ers,Flyers[note 13]
7,Boston,4794447,Patriots[note 14],Red Sox[note 15],Celtics,Bruins
8,Minneapolis–Saint Paul,3551036,Vikings,Twins,Timberwolves,Wild[note 16]
9,Denver,2853077,Broncos,Rockies,Nuggets[note 17],Avalanche[note 18]


**DATA CLEANING**

In [4]:
def split_nhl_name(row):
    row["NHL"] = re.findall("([A-Z][a-z]*)+?", row["NHL"])
    return row 

In [5]:
cities.rename(columns = {"Population (2016 est.)[8]": "Population"}, inplace = True)
cities["NHL"] = cities["NHL"].replace("\[.*", "", regex = True).replace({"—":np.nan, "":np.nan})
cities = cities[["Metropolitan area", "Population", "NHL"]].dropna()
cities = cities.apply(lambda row: split_nhl_name(row), axis = 1)
cities = cities.explode(column = "NHL")
cities["Population"] = pd.to_numeric(cities["Population"])
cities

,Metropolitan area,Population,NHL
0,New York City,20153634,Rangers
0,New York City,20153634,Islanders
0,New York City,20153634,Devils
1,Los Angeles,13310447,Kings
1,Los Angeles,13310447,Ducks
2,San Francisco Bay Area,6657982,Sharks
3,Chicago,9512999,Blackhawks
4,Dallas–Fort Worth,7233323,Stars
5,"Washington, D.C.",6131977,Capitals
6,Philadelphia,6070500,Flyers


In [6]:
nhl_df=pd.read_csv("assets/nhl.csv")
nhl_df = nhl_df[nhl_df["year"] == 2018].drop([0,9,18,26])
nhl_df.rename(columns = {"team": "NHL"}, inplace = True)
nhl_df["NHL"] = nhl_df["NHL"].replace([".*\s", "\*"], "", regex = True)
nhl_df["W"] = pd.to_numeric(nhl_df["W"])
nhl_df["L"] = pd.to_numeric(nhl_df["L"])
nhl_df["Win/Loss"] = nhl_df["W"] / (nhl_df["L"] + nhl_df["W"])
nhl_df = nhl_df[["NHL", "W", "L", "Win/Loss"]]
nhl_df

,NHL,W,L,Win/Loss
1,Lightning,54,23,0.701299
2,Bruins,50,20,0.714286
3,Leafs,49,26,0.653333
4,Panthers,44,30,0.594595
5,Wings,30,39,0.434783
6,Canadiens,29,40,0.420290
7,Senators,28,43,0.394366
8,Sabres,25,45,0.357143
10,Capitals,49,26,0.653333
11,Penguins,47,29,0.618421


**MERGING**

In [7]:
merged_df = pd.merge(cities, nhl_df, how = "inner", on = "NHL")
population_by_region = merged_df.groupby("Metropolitan area").agg({"Population":np.nanmean}).sort_index()
win_loss_by_region = merged_df.groupby("Metropolitan area").agg({"Win/Loss":np.nanmean}).sort_index()
population_by_region = population_by_region["Population"] # pass in metropolitan area population from cities
win_loss_by_region = win_loss_by_region["Win/Loss"] # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

**Correlation**

In [8]:
stats.pearsonr(population_by_region, win_loss_by_region)

PearsonRResult(statistic=0.012486162921209881, pvalue=0.9497182859911808)

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [9]:
def split_nba_name(row):
    row["NBA"] = re.findall("([A-Z0-9][0-9a-z]*)+?", row["NBA"])
    return row

In [10]:
def nba_correlation():
    # cleaned data from assets/wikipedia_data.html
    
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities.rename(columns = {"Population (2016 est.)[8]": "Population"}, inplace = True)
    cities["NBA"] = cities["NBA"].replace("\[.*", "", regex = True).replace({"—":np.nan, "":np.nan})
    cities = cities[["Metropolitan area", "Population", "NBA"]].dropna()
    cities = cities.apply(lambda row: split_nba_name(row), axis = 1)
    cities = cities.explode(column = "NBA")
    cities["Population"] = pd.to_numeric(cities["Population"])

    # cleaned data from assets/nba.csv
    
    nba_df=pd.read_csv("assets/nba.csv")
    nba_df = nba_df[nba_df["year"] == 2018]
    nba_df.rename(columns = {"team": "NBA"}, inplace = True)
    nba_df["NBA"] = nba_df["NBA"].str.extract("(?:^.* )([\w]*)")
    nba_df["W/L%"] = pd.to_numeric(nba_df["W/L%"])
    nba_df = nba_df[["NBA", "W/L%",]]
    
    # merged data
        
    merged_df = pd.merge(cities, nba_df, how = "inner", on = "NBA")
    win_loss_by_region = merged_df.groupby("Metropolitan area").agg({"W/L%":np.nanmean}).sort_index()
    population_by_region = merged_df.groupby("Metropolitan area").agg({"Population":np.nanmean}).sort_index()
    
    # Final
    
    population_by_region = population_by_region["Population"] # pass in metropolitan area population from cities
    win_loss_by_region = win_loss_by_region["W/L%"] # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]
    
    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nba_correlation()

-0.17636350642182935

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [11]:
def split_mlb_name(row):
    if row["MLB"] == "Cubs White Sox":
        row["MLB"] = ["Cubs", "White Sox"]
    elif row["MLB"] == "Red Sox":
        row["MLB"] = ["Red Sox"]
    elif row["MLB"] == "Blue Jays":
        row["MLB"] = ["Blue Jays"]
    else:
        row["MLB"] = re.findall("([A-Z][a-z]*)+?", row["MLB"])
    return row

def clean_mlb_name(row):
    if row["MLB"] == "Boston Red Sox":
        row["MLB"] = "Red Sox"
    elif row["MLB"] == "Chicago White Sox":
        row["MLB"] = "White Sox"
    elif row["MLB"] == "Toronto Blue Jays":
        row["MLB"] = "Blue Jays"
    else:
        m = re.match("(?:^.* )([\w]*)", row["MLB"])
        row["MLB"] = m.group(1) 
    return row

In [12]:
def mlb_correlation(): 
    # cleaned data from assets/wikipedia_data.html
    
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities.rename(columns = {"Population (2016 est.)[8]": "Population"}, inplace = True)

    cities["MLB"] = cities["MLB"].replace("\[.*", "", regex = True).replace({"—":np.nan, "":np.nan})
    cities = cities[["Metropolitan area", "Population", "MLB"]].dropna()
    cities = cities.apply(lambda row: split_mlb_name(row), axis = 1)
    cities = cities.explode(column = "MLB")
    cities["Population"] = pd.to_numeric(cities["Population"])
    
    # cleaned data from assests/mlb.csv
    
    mlb_df=pd.read_csv("assets/mlb.csv")
    mlb_df = mlb_df[mlb_df["year"] == 2018]
    mlb_df.rename(columns = {"team": "MLB"}, inplace = True)

    mlb_df = mlb_df.apply(lambda row: clean_mlb_name(row), axis = 1)
    mlb_df["W-L%"] = pd.to_numeric(mlb_df["W-L%"])
    mlb_df = mlb_df[["MLB", "W-L%"]]
    
    # merged data
    
    merged_df = pd.merge(cities, mlb_df, how = "inner", on = "MLB" )
    population_by_region = merged_df.groupby("Metropolitan area").agg({"Population":np.nanmean}).sort_index()
    win_loss_by_region = merged_df.groupby("Metropolitan area").agg({"W-L%":np.nanmean}).sort_index()
    
    # final answer

    population_by_region = population_by_region["Population"] # pass in metropolitan area population from cities
    win_loss_by_region = win_loss_by_region["W-L%"] # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

mlb_correlation()

0.15003737475409495

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [13]:
def split_nfl_name(row):
    row["NFL"] = re.findall("([0-9A-Z][0-9a-z]*)+?", row["NFL"])
    return row

In [14]:
def nfl_correlation(): 
    # cleaned data from assets/wikipedia_data.html
    
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities.rename(columns = {"Population (2016 est.)[8]": "Population"}, inplace = True)

    cities["NFL"] = cities["NFL"].replace("\[.*", "", regex = True).str.strip().replace({"—":np.nan, "":np.nan})
    cities = cities[["Metropolitan area", "Population", "NFL"]].dropna()
    cities = cities.apply(lambda row: split_nfl_name(row), axis = 1)
    cities = cities.explode(column = "NFL")
    cities["Population"] = pd.to_numeric(cities["Population"])
    
    # cleaned data from assets/nfl_df.csv
    
    nfl_df=pd.read_csv("assets/nfl.csv")
    nfl_df = nfl_df[nfl_df["year"] == 2018]
    nfl_df.rename(columns = {"team": "NFL"}, inplace = True)
    nfl_df = nfl_df[["NFL", "W-L%"]].drop([0, 5,10,15,20,25,30,35])
    nfl_df["NFL"] = nfl_df["NFL"].str.extract("(?:^.* )([\w]*)")
    nfl_df["W-L%"] = pd.to_numeric(nfl_df["W-L%"])
    
    # merged data
    
    merged_df = pd.merge(cities, nfl_df, how = "inner", on = "NFL" )
    population_by_region = merged_df.groupby("Metropolitan area").agg({"Population":np.nanmean}).sort_index()
    win_loss_by_region = merged_df.groupby("Metropolitan area").agg({"W-L%":np.nanmean}).sort_index()
    
    # final answer
    
    population_by_region = population_by_region["Population"] # pass in metropolitan area population from cities
    win_loss_by_region = win_loss_by_region["W-L%"] # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]

nfl_correlation()

0.004282141436393022

## Question 5
Hypothesis - **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**.  
Using t-tests ([`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports.  
Are there any sports where we can reject the null hypothesis?

**NHL**

In [15]:
def split_nhl_name(row):
    row["NHL"] = re.findall("([A-Z][a-z]*)+?", row["NHL"])
    return row    
    
def nhl(): 
    # cleaned data from assets/wikipedia_data.html
    
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]

    cities.rename(columns = {"Population (2016 est.)[8]": "Population"}, inplace = True)
    cities["NHL"] = cities["NHL"].replace("\[.*" , "", regex = True).replace({"—":np.nan, "":np.nan})
    cities = cities[["Metropolitan area", "Population", "NHL"]].dropna()
    cities = cities.apply(lambda row: split_nhl_name(row), axis = 1)
    cities = cities.explode(column = "NHL")
    cities["Population"] = pd.to_numeric(cities["Population"])

    # cleaned data from assets/nhl.csv

    nhl_df=pd.read_csv("assets/nhl.csv")
    nhl_df = nhl_df[nhl_df["year"] == 2018].drop([0,9,18,26])

    nhl_df.rename(columns = {"team": "NHL"}, inplace = True)
    nhl_df["NHL"] = nhl_df["NHL"].replace([".*\s", "\*"], "", regex = True)

    nhl_df["W"] = pd.to_numeric(nhl_df["W"])
    nhl_df["L"] = pd.to_numeric(nhl_df["L"])
    nhl_df["Win/Loss"] = nhl_df["W"] / (nhl_df["L"] + nhl_df["W"])

    nhl_df = nhl_df[["NHL", "W", "L", "Win/Loss"]]

    # merged data
    
    merged_df = pd.merge(cities, nhl_df, how = "inner", on = "NHL")
    win_loss_by_region = merged_df.groupby("Metropolitan area").agg({"Win/Loss":np.nanmean}).rename(columns = {"Win/Loss":"NHL"})
    
    return win_loss_by_region

**NBA**

In [16]:
def split_nba_name(row):
    row["NBA"] = re.findall("([A-Z0-9][0-9a-z]*)+?", row["NBA"])
    return row

def nba():
    # cleaned data from assets/wikipedia_data.html
    
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities.rename(columns = {"Population (2016 est.)[8]": "Population"}, inplace = True)
    cities["NBA"] = cities["NBA"].replace("\[.*", "", regex = True).replace({"—":np.nan, "":np.nan})
    cities = cities[["Metropolitan area", "Population", "NBA"]].dropna()
    cities = cities.apply(lambda row: split_nba_name(row), axis = 1)
    cities = cities.explode(column = "NBA")
    cities["Population"] = pd.to_numeric(cities["Population"])

    # cleaned data from assets/nba.csv
    
    nba_df=pd.read_csv("assets/nba.csv")
    nba_df = nba_df[nba_df["year"] == 2018]
    nba_df.rename(columns = {"team": "NBA"}, inplace = True)
    nba_df["NBA"] = nba_df["NBA"].str.extract("(?:^.* )([\w]*)")
    nba_df["W/L%"] = pd.to_numeric(nba_df["W/L%"])
    nba_df = nba_df[["NBA", "W/L%",]]
    
    # merged data
        
    merged_df = pd.merge(cities, nba_df, how = "inner", on = "NBA")
    win_loss_by_region = merged_df.groupby("Metropolitan area").agg({"W/L%":np.nanmean}).rename(columns = {"W/L%": "NBA"})

    return win_loss_by_region

**MLB**

In [17]:
def split_mlb_name(row):
    if row["MLB"] == "Cubs White Sox":
        row["MLB"] = ["Cubs", "White Sox"]
    elif row["MLB"] == "Red Sox":
        row["MLB"] = ["Red Sox"]
    elif row["MLB"] == "Blue Jays":
        row["MLB"] = ["Blue Jays"]
    else:
        row["MLB"] = re.findall("([A-Z][a-z]*)+?", row["MLB"])
    return row

def clean_mlb_name(row):
    if row["MLB"] == "Boston Red Sox":
        row["MLB"] = "Red Sox"
    elif row["MLB"] == "Chicago White Sox":
        row["MLB"] = "White Sox"
    elif row["MLB"] == "Toronto Blue Jays":
        row["MLB"] = "Blue Jays"
    else:
        m = re.match("(?:^.* )([\w]*)", row["MLB"])
        row["MLB"] = m.group(1) 
    return row

def mlb(): 
    # cleaned data from assets/wikipedia_data.html
    
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities.rename(columns = {"Population (2016 est.)[8]": "Population"}, inplace = True)

    cities["MLB"] = cities["MLB"].replace("\[.*", "", regex = True).replace({"—":np.nan, "":np.nan})
    cities = cities[["Metropolitan area", "Population", "MLB"]].dropna()
    cities = cities.apply(lambda row: split_mlb_name(row), axis = 1)
    cities = cities.explode(column = "MLB")
    cities["Population"] = pd.to_numeric(cities["Population"])
    
    # cleaned data from assests/mlb.csv
    
    mlb_df=pd.read_csv("assets/mlb.csv")
    mlb_df = mlb_df[mlb_df["year"] == 2018]
    mlb_df.rename(columns = {"team": "MLB"}, inplace = True)

    mlb_df = mlb_df.apply(lambda row: clean_mlb_name(row), axis = 1)
    mlb_df["W-L%"] = pd.to_numeric(mlb_df["W-L%"])
    mlb_df = mlb_df[["MLB", "W-L%"]]
    
    # merged data
    
    merged_df = pd.merge(cities, mlb_df, how = "inner", on = "MLB" )
    win_loss_by_region = merged_df.groupby("Metropolitan area").agg({"W-L%":np.nanmean}).rename(columns = {"W-L%":"MLB"})
    
    return win_loss_by_region

**NFL**

In [18]:
def split_nfl_name(row):
    row["NFL"] = re.findall("([0-9A-Z][0-9a-z]*)+?", row["NFL"])
    return row

def nfl(): 
    # cleaned data from assets/wikipedia_data.html
    
    cities=pd.read_html("assets/wikipedia_data.html")[1]
    cities=cities.iloc[:-1,[0,3,5,6,7,8]]
    cities.rename(columns = {"Population (2016 est.)[8]": "Population"}, inplace = True)

    cities["NFL"] = cities["NFL"].replace("\[.*", "", regex = True).str.strip().replace({"—":np.nan, "":np.nan})
    cities = cities[["Metropolitan area", "Population", "NFL"]].dropna()
    cities = cities.apply(lambda row: split_nfl_name(row), axis = 1)
    cities = cities.explode(column = "NFL")
    cities["Population"] = pd.to_numeric(cities["Population"])
    
    # cleaned data from assets/nfl_df.csv
    
    nfl_df=pd.read_csv("assets/nfl.csv")
    nfl_df = nfl_df[nfl_df["year"] == 2018]
    nfl_df.rename(columns = {"team": "NFL"}, inplace = True)
    nfl_df = nfl_df[["NFL", "W-L%"]].drop([0, 5,10,15,20,25,30,35])
    nfl_df["NFL"] = nfl_df["NFL"].str.extract("(?:^.* )([\w]*)")
    nfl_df["W-L%"] = pd.to_numeric(nfl_df["W-L%"])
    
    # merged data
    
    merged_df = pd.merge(cities, nfl_df, how = "inner", on = "NFL" )
    win_loss_by_region = merged_df.groupby("Metropolitan area").agg({"W-L%":np.nanmean}).rename(columns = {"W-L%": "NFL"})

    return win_loss_by_region

In [19]:
import itertools

def sports_team_performance():
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    d = {'NFL': nfl(), 'NBA': nba(), 'NHL': nhl(), 'MLB': mlb()}
    p_values = pd.DataFrame({k:np.nan for k in sports}, index=sports)
    
    for col1, col2 in itertools.combinations(sports, 2):
        merged_df = pd.merge(d[col1], d[col2], how = "inner", left_index = True, right_index = True)
        # print("{}-{}:{}".format(col1,col2, len(merged_df)))
        p_values.loc[col1, col2] = stats.ttest_rel(merged_df[col1], merged_df[col2])[1]
        p_values.loc[col2, col1] = p_values.loc[col1, col2]
    return p_values

sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.937509,0.030318,0.803459
NBA,0.937509,NaN,0.022386,0.949566
NHL,0.030318,0.022386,NaN,0.000703
MLB,0.803459,0.949566,0.000703,NaN


**We can reject the hypothesis for pairs NHL-NBA, NHL-NFL, MLB-NHL as p-value is less than 0.05**

**95% Confidence Interval**

In [20]:
merged_df = pd.merge(nhl(), nba(), how = "inner", left_index = True, right_index = True)
stats.ttest_rel(merged_df['NHL'], merged_df['NBA']).confidence_interval(confidence_level=0.95)

ConfidenceInterval(low=0.011860436003066965, high=0.13086859619800043)